## Les pipelines
Laurent Cetinsoy

A quoi ça sert ? Bah je sais pas, c'est le but de ce notebook. 


Une pipeline est une chaîne de traitement, découpées en petites étapes ("steps") intermédiares. Une fois une pipeline définie il est possible de les publier et de les lancer de manière régulière comme avec Airflow ou un cron ou à chaque fois qu'un événement donné se produit. 

Tout ce les trucs liés aux pipelines sont dans le module azureml.pipeline. 

Reférences utiles :
- https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core?view=azure-ml-py



Un objet de type Pipeline représente une pipeline (sans dec) et est constitué de plusieurs PipelineStep.
Quelles sont les principaux types de Pipeline ? 

https://docs.microsoft.com/fr-fr/azure/machine-learning/concept-ml-pipelines

Quelles sont les principales méthodes de la classe Pipeline ? 

Quels sont les arguments du constructeur de la classe Pipeline (cf : https://docs.microsoft.com/fr-fr/python/api/azureml-pipeline-core/azureml.pipeline.core.pipeline(class)?view=azure-ml-py)

Quelles sont les principales classes filles de la classe PipelineStep ?

cf : https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.pipelinestep?view=azure-ml-py

Quelles sont les principales méthodes de la classe PipelineStep et de ses classes filles ? 

In [ ]:
#chargement workspace
from azureml.core import Workspace
ws = Workspace.from_config("config.json")

In [ ]:
#chargement comptue target si existe et création sinon
from azureml.core.compute import AmlCompute, ComputeTarget
from azureml.core.compute_target import ComputeTargetException

try:
    cpu_cluster = ComputeTarget(ws,"elkalkul")
    print("CT existante")
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    cpu_cluster = ComputeTarget.create(ws, "elkalkul", compute_config)
    print("CT créée")

## Création, publication et orchestration de notre première pipeline

### PythonScriptStep


Avec la classe PythonScriptStep, faire un step qui execute le script coucou.py que vous aurez créé et qui affiche "coucou" 

In [ ]:
from azureml.pipeline.steps import PythonScriptStep
test_step = PythonScriptStep(
    script_name="coucou.py",
    source_directory=".",
    compute_target=cpu_cluster
)

Si vous avez déjà créé une AciTargetCompute, récupérer une référence dessus. Sinon en créé un 

In [ ]:
#fait plus haut

Créer une ComputeTarget qui pointe vers votre AciTargetCompute

In [ ]:
#fait plus haut

Créer une pipeline qui contient le step précédent

In [ ]:
from azureml.pipeline.core import Pipeline
first_pipe = Pipeline(ws, steps=[test_step])

Publier la pipeline

In [ ]:
published_pipe = first_pipe.publish(name="pipe_2", description="My 2nd pipeline")

Afficher la liste de toutes les Pipelines "publiées" de votre workspace avec la méthode **list** de la classe PublishedPipeline.

In [ ]:
published_pipe.list(ws)

Créer une experience et soumettre la pipeline. Vérifier que la pipeline s'est executée sur le portail azure

In [ ]:
from azureml.core.experiment import Experiment
pipe_exp = Experiment(ws, "pipeline_experiment_2")
pipe_run = pipe_exp.submit(published_pipe)

Quelle différence entre une pipeline et une pipeline publiée ?

Faire en sorte que la pipeline s'execute toutes les minutes avec la classe Schedule (cf https://docs.microsoft.com/en-us/python/api/azureml-pipeline-core/azureml.pipeline.core.schedule(class)?view=azure-ml-py)

In [ ]:
from azureml.pipeline.core import Schedule, ScheduleRecurrence
recurrence = ScheduleRecurrence(frequency="Minute", interval=1)
schedule = Schedule.create(ws,
                           name="TestSchedule",
                           pipeline_id="56bef8a5-58d4-4d2d-b73e-090a7a8eba73",
                           experiment_name="pipeline_experiment_2",
                           recurrence=recurrence)
type(schedule)

In [ ]:
Schedule.list(ws)

In [ ]:
for s in Schedule.list(ws):
    s.disable()
    
Schedule.list(ws)

Bonus : Modifier la pipeline pour qu'elle fasse quelque chose d'un peu plus utile (entraînement d'un modèle simple avec register par exemple ou upload d'un dataset)

In [ ]:
from azureml.core import Environment 
Environment.list(ws).keys()

In [ ]:
from azureml.core.runconfig import RunConfiguration
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core import Environment 

aml_run_config = RunConfiguration()
aml_run_config.target = cpu_cluster

# Si on veut un environnement déjà existant
# pour récupérer la liste des environnements:
# Environment.list(ws).keys()
USE_CURATED_ENV = True
if USE_CURATED_ENV :
    curated_environment = Environment.get(workspace=ws, name="AzureML-Tutorial")
    aml_run_config.environment = curated_environment
    
#sinon on choisit soi même ses packages, par exemple
else:
    aml_run_config.environment.python.user_managed_dependencies = False
    aml_run_config.environment.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pandas','scikit-learn'], 
    pip_packages=['azureml-sdk', 'azureml-dataprep[fuse,pandas]'], 
    pin_sdk_version=False)

In [ ]:
from azureml.pipeline.steps import PythonScriptStep
reg_step = PythonScriptStep(
    script_name="train.py",
    source_directory=".",
    runconfig=aml_run_config,
    compute_target=cpu_cluster
)

In [ ]:
from azureml.pipeline.core import Pipeline
reg_pipe = Pipeline(ws, steps=[reg_step])

Republier la pipeline et l'executer

In [ ]:
published_pipe = reg_pipe.publish(name="reg_pipe", description="My regression pipeline")

In [ ]:
from azureml.core.experiment import Experiment
pipe_exp = Experiment(ws, "exp_reg_pipe")
pipe_run = pipe_exp.submit(published_pipe)